<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=5cc77256433cd6f9461849f22624416fd4813c6a26349515abf2826ea259abd2
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.1
    Uninstalling protobuf-6.33.1:
      Successfully uninstalled protobuf-6.33.1


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-12-05 11:20:07
-------------------
qualified stocks: 86
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.28 C
CY Investment:  1.55 C
Reserve:  4.95 L
Current:  1.41 C
-------------------
Today PnL: -2.57 L (-1.79%)
Current PnL: -26.86 L (-17.35%)
CY Booked + Current PnL: -12.57 L (-8.12%)
-------------------
Total profit:  1.39 L
Total loss:  -28.26 L
-------------------
Total Booked + Current PnL: 14.41 L (11.27%)
Total Booked PnL: 41.27 L (32.29%)
Curr Year Booked PnL: 14.29 L (10.11%)
Prev Year Booked PnL: 26.98 L (21.11%)
Est FTT:  2.33 C
Est FTT PnL: 91.49 L (64.7%)
Deployed:  1.28 C
Current:  1.41 C
CAGR/XIRR %: 5.73%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,INDIGOPNTS,1408.00,-0.33,-8.98,9.89,0.02,15713.0,-15678.0,158881.0,142.19,65.0,M-SC,15.59,234.0,-1.00,1.16,38.20,OX40N,NTT,PAINTS
77,TTKPRESTIG,770.00,-0.11,-16.80,20.30,0.09,17021.0,-16930.0,83847.0,94.66,37.0,M-SC,2.75,253.0,-0.99,0.61,10.16,OX40N,NTT,DURABLES
50,MASFIN,398.61,0.05,-1.96,24.49,22.05,23525.0,-1920.0,96060.0,-15.54,60.0,H-SC,7.02,164.0,-0.08,0.70,39.63,XR,ATH,FINANCE
43,ITC,452.00,0.07,-1.45,12.06,10.44,23787.0,-2900.0,197238.0,-41.15,46.0,X-LC,1.58,5.0,-0.12,1.45,4.03,X40,NTT,FMCG
35,ICICIGI,2252.93,0.37,5.61,13.81,20.19,25425.0,9783.0,184103.0,-17.20,47.0,X-MC,6.61,68.0,0.38,1.35,21.57,X40,ATH,INSURANCE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
80,VALIANTORG,838.00,5.80,-43.86,210.20,74.15,193071.0,-71754.0,91851.0,-265.32,45.0,H-SC,19.96,149.0,-0.37,0.67,17.74,XR,NTT,CHEMICALS
32,HCLTECH,1922.01,2.80,9.44,13.97,24.73,36988.0,22841.0,264765.0,14.46,71.0,X-LC,5.26,7.0,0.62,1.94,26.12,X40,ATH,IT
73,TCS,4389.96,2.54,-11.40,34.63,19.28,106143.0,-39450.0,306506.0,-22.65,72.0,X-LC,6.51,1.0,-0.37,2.25,13.31,X40,ATH,IT
29,HAPPSTMNDS,1480.71,2.16,-36.59,191.54,84.85,155653.0,-46901.0,81264.0,-24.68,53.0,H-SC,14.03,147.0,-0.30,0.60,4.30,AR,ATH,IT
41,INFY,2275.00,1.78,10.69,39.91,54.87,138223.0,33450.0,346338.0,-13.85,72.0,X-LC,3.27,2.0,0.24,2.54,19.86,X40,BTT,IT


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
15,CAMS,4762.00,-80.11,-79.88,520.38,24.79,263635.0,-201194.0,50662.0,-80.87,7.0,X-SC,1.60,86.0,-0.76,0.37,0.00,X40N,NTT,MISC
44,JCHAC,2282.00,-4.88,-34.73,53.19,-0.01,44373.0,-44382.0,83423.0,16965.22,8.0,M-SC,1.01,233.0,-1.00,0.61,0.00,OX40N,NTT,AC
33,HINDUNILVR,2922.00,-4.66,-8.68,24.47,13.67,57443.0,-22313.0,234750.0,-15.44,33.0,X-LC,3.31,14.0,-0.39,1.72,10.82,XY25,NTT,FMCG
51,MEDANTA,1486.00,-3.90,-2.72,27.54,24.08,34012.0,-3449.0,123501.0,-10.62,33.0,X-SC,6.01,89.0,-0.10,0.91,16.37,XY24,NTT,HEALTHCARE
61,ROUTE,2227.21,-3.24,-48.27,226.81,69.05,156118.0,-64234.0,68832.0,-58.05,46.0,H-SC,23.71,140.0,-0.41,0.50,6.15,SR,ATH,IT


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
37,IEX,219.0,-1.63,-0.6,49.52,48.63,98426.0,-1194.0,198760.0,-33.87,60.0,H-SC,14.75,136.0,-0.01,1.46,10.89,XR,NTT,MISC


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,-0.94,-6.93,123.39,107.91,169559.0,-10227.0,137417.0,-24.20,30.0,M-SC,11.25,216.0,-0.06,1.01,0.00,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,-0.33,-8.98,9.89,0.02,15713.0,-15678.0,158881.0,142.19,65.0,M-SC,15.59,234.0,-1.00,1.16,38.20,OX40N,NTT,PAINTS
77,TTKPRESTIG,770.00,-0.11,-16.80,20.30,0.09,17021.0,-16930.0,83847.0,94.66,37.0,M-SC,2.75,253.0,-0.99,0.61,10.16,OX40N,NTT,DURABLES
47,KANSAINER,340.00,-2.78,-24.51,50.31,13.47,102421.0,-66087.0,203580.0,-68.90,29.0,H-SC,2.41,158.0,-0.65,1.49,3.51,XY24,NTT,PAINTS
72,TATAELXSI,9161.00,1.17,-24.55,75.00,32.03,74599.0,-32370.0,99465.0,-18.39,47.0,H-SC,7.86,148.0,-0.43,0.73,11.41,OX40N,NTT,IT


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.0,1.01,6.44,61.84,72.27,111538.0,10918.0,180366.0,-8.8,72.0,M-LC,2.95,99.0,0.1,1.32,13.02,XR,NTT,IT


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.00,1.01,6.44,61.84,72.27,111538.0,10918.0,180366.0,-8.80,72.0,M-LC,2.95,99.0,0.10,1.32,13.02,XR,NTT,IT
38,INDIAMART,4810.62,-1.28,-1.18,109.18,106.72,133073.0,-1452.0,121884.0,-51.01,36.0,H-SC,1.56,139.0,-0.01,0.89,21.93,AR,ATH,MISC
37,IEX,219.00,-1.63,-0.60,49.52,48.63,98426.0,-1194.0,198760.0,-33.87,60.0,H-SC,14.75,136.0,-0.01,1.46,10.89,XR,NTT,MISC
85,ZYDUSLIFE,1286.17,-0.09,-1.50,37.69,35.63,77806.0,-3138.0,206436.0,-15.41,41.0,H-MC,3.42,119.0,-0.04,1.51,14.93,AR,ATH,PHARMA
25,FINCABLES,1641.55,-0.94,-6.93,123.39,107.91,169559.0,-10227.0,137417.0,-24.20,30.0,M-SC,11.25,216.0,-0.06,1.01,0.00,OX40N,ATH,CABLES


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
15,CAMS,4762.00,-80.11,-79.88,520.38,24.79,263635.0,-201194.0,50662.0,-80.87,7.0,X-SC,1.60,86.0,-0.76,0.37,0.00,X40N,NTT,MISC
9,BAJAJHFL,181.50,-0.54,-21.11,88.18,48.45,139481.0,-42328.0,158178.0,-26.75,12.0,X-MC,9.41,64.0,-0.30,1.16,0.00,X40N,ATH,FINANCE
11,BATAINDIA,2096.00,-1.47,-37.87,117.63,35.20,94030.0,-48733.0,79937.0,4.84,23.0,X-SC,14.42,92.0,-0.52,0.59,0.00,X40,NTT,FOOTWEAR
22,DIXON,18931.72,-2.90,-12.59,39.09,21.58,47885.0,-17640.0,122499.0,-55.95,24.0,X-MC,5.77,56.0,-0.37,0.90,8.45,X40N,ATH,IT
54,PGHH,17907.65,-0.31,-5.89,42.12,33.76,79607.0,-11820.0,189000.0,-33.58,28.0,X-MC,4.68,57.0,-0.15,1.38,0.00,X40,ATH,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
3,ACC,3906.0,-1.37,-23.72,115.21,64.15,209106.0,-56451.0,181500.0,-55.44,41.0,X-SC,1.49,82.0,-0.27,1.33,1.81,XY24,BTT,CEMENT
8,AWL,485.0,0.89,-20.96,94.19,53.50,224654.0,-63240.0,238511.0,-61.72,31.0,X-MC,1.56,58.0,-0.28,1.75,5.94,XY24,NTT,FMCG
43,ITC,452.0,0.07,-1.45,12.06,10.44,23787.0,-2900.0,197238.0,-41.15,46.0,X-LC,1.58,5.0,-0.12,1.45,4.03,X40,NTT,FMCG
15,CAMS,4762.0,-80.11,-79.88,520.38,24.79,263635.0,-201194.0,50662.0,-80.87,7.0,X-SC,1.60,86.0,-0.76,0.37,0.00,X40N,NTT,MISC
36,ICICIPRULI,790.0,1.49,2.84,27.35,30.97,49712.0,5025.0,181763.0,-21.38,54.0,X-MC,2.25,75.0,0.10,1.33,15.76,X40,ATH,INSURANCE


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
11,BATAINDIA,2096.00,-1.47,-37.87,117.63,35.20,94030.0,-48733.0,79937.0,4.84,23.0,X-SC,14.42,92.0,-0.52,0.59,0.0,X40,NTT,FOOTWEAR
15,CAMS,4762.00,-80.11,-79.88,520.38,24.79,263635.0,-201194.0,50662.0,-80.87,7.0,X-SC,1.60,86.0,-0.76,0.37,0.0,X40N,NTT,MISC
9,BAJAJHFL,181.50,-0.54,-21.11,88.18,48.45,139481.0,-42328.0,158178.0,-26.75,12.0,X-MC,9.41,64.0,-0.30,1.16,0.0,X40N,ATH,FINANCE
55,QUESS,424.00,-2.25,-31.69,109.10,42.84,48448.0,-20599.0,44407.0,-55.37,34.0,X-SC,37.35,83.0,-0.43,0.33,0.0,XY24,NTT,MISC
54,PGHH,17907.65,-0.31,-5.89,42.12,33.76,79607.0,-11820.0,189000.0,-33.58,28.0,X-MC,4.68,57.0,-0.15,1.38,0.0,X40,ATH,FMCG


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,QUESS,424.00,-2.25,-31.69,109.10,42.84,48448.0,-20599.0,44407.0,-55.37,34.0,X-SC,37.35,83.0,-0.43,0.33,0.00,XY24,NTT,MISC
15,CAMS,4762.00,-80.11,-79.88,520.38,24.79,263635.0,-201194.0,50662.0,-80.87,7.0,X-SC,1.60,86.0,-0.76,0.37,0.00,X40N,NTT,MISC
59,RELAXO,1176.00,-0.36,-47.59,190.59,52.31,145008.0,-69076.0,76084.0,-44.25,34.0,X-SC,5.38,91.0,-0.48,0.56,1.84,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,-1.47,-37.87,117.63,35.20,94030.0,-48733.0,79937.0,4.84,23.0,X-SC,14.42,92.0,-0.52,0.59,0.00,X40,NTT,FOOTWEAR
22,DIXON,18931.72,-2.90,-12.59,39.09,21.58,47885.0,-17640.0,122499.0,-55.95,24.0,X-MC,5.77,56.0,-0.37,0.90,8.45,X40N,ATH,IT


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4389.96,2.54,-11.40,34.63,19.28,106143.0,-39450.0,306506.0,-22.65,72.0,X-LC,6.51,1.0,-0.37,2.25,13.31,X40,ATH,IT
41,INFY,2275.00,1.78,10.69,39.91,54.87,138223.0,33450.0,346338.0,-13.85,72.0,X-LC,3.27,2.0,0.24,2.54,19.86,X40,BTT,IT
75,TMPV,600.00,-0.21,-15.71,68.68,42.18,160013.0,-43426.0,232984.0,-24.30,31.0,X-LC,3.95,3.0,-0.27,1.71,0.92,XY24,NTT,AUTO
81,VBL,671.64,-0.52,-4.05,41.40,35.67,125463.0,-12792.0,303050.0,-15.46,56.0,X-LC,5.33,4.0,-0.10,2.22,9.24,X40N,ATH,FMCG
43,ITC,452.00,0.07,-1.45,12.06,10.44,23787.0,-2900.0,197238.0,-41.15,46.0,X-LC,1.58,5.0,-0.12,1.45,4.03,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
40,INDUSINDBK,1800.00,1.36,-36.62,109.68,32.89,53669.0,-28274.0,48932.0,-703.98,63.0,L-MC,6.20,259.0,-0.53,0.36,34.75,XR,NTT,BANKS
6,ASIANTILES,137.00,-2.30,-17.03,118.95,81.67,92587.0,-15973.0,77837.0,6994.44,38.0,L-SC,16.28,271.0,-0.17,0.57,50.73,XR,NTT,CERAMICS
50,MASFIN,398.61,0.05,-1.96,24.49,22.05,23525.0,-1920.0,96060.0,-15.54,60.0,H-SC,7.02,164.0,-0.08,0.70,39.63,XR,ATH,FINANCE
13,BSOFT,831.70,0.27,-20.76,90.28,50.79,100626.0,-29193.0,111460.0,1.66,75.0,H-SC,5.20,151.0,-0.29,0.82,29.78,XR,ATH,IT
39,INDIGOPNTS,1408.00,-0.33,-8.98,9.89,0.02,15713.0,-15678.0,158881.0,142.19,65.0,M-SC,15.59,234.0,-1.00,1.16,38.20,OX40N,NTT,PAINTS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,INDIGOPNTS,1408.00,-0.33,-8.98,9.89,0.02,15713.0,-15678.0,158881.0,142.19,65.0,M-SC,15.59,234.0,-1.00,1.16,38.20,OX40N,NTT,PAINTS
13,BSOFT,831.70,0.27,-20.76,90.28,50.79,100626.0,-29193.0,111460.0,1.66,75.0,H-SC,5.20,151.0,-0.29,0.82,29.78,XR,ATH,IT
5,ASIANPAINT,3460.25,-0.43,2.91,17.52,20.95,45396.0,7339.0,259107.0,1.90,76.0,X-LC,18.43,31.0,0.16,1.90,39.59,X40,ATH,PAINTS
41,INFY,2275.00,1.78,10.69,39.91,54.87,138223.0,33450.0,346338.0,-13.85,72.0,X-LC,3.27,2.0,0.24,2.54,19.86,X40,BTT,IT
32,HCLTECH,1922.01,2.80,9.44,13.97,24.73,36988.0,22841.0,264765.0,14.46,71.0,X-LC,5.26,7.0,0.62,1.94,26.12,X40,ATH,IT


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.76
1,25,45.64
2,50,76.57


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    45.14
MC    30.07
LC    24.78
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     28.35
X40      22.43
X40N     10.81
XR       10.07
AR        9.43
XY25      9.31
OX40N     7.77
SR        1.02
MH        0.80
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    24.59
X-MC    23.35
X-LC    21.20
M-SC    12.20
X-SC     6.95
H-MC     4.94
M-MC     1.42
L-SC     1.40
M-LC     1.32
H-LC     1.23
L-LC     1.03
L-MC     0.36
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,17.30,-6.59,41.06
IT,12.85,-15.04,77.23
FINANCE,9.60,-18.34,68.35
ELECTRICAL,6.27,-9.78,50.00
PAINTS,6.20,-9.03,25.23
MISC,6.17,-54.48,117.76
INSURANCE,4.52,-2.36,38.44
PHARMA,4.07,-0.40,32.53
AUTO,3.45,-21.93,74.22


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3168790.0,21
XR,1285850.0,13
AR,1285293.0,10
X40,978513.0,14
X40N,974224.0,9
OX40N,730028.0,10
XY25,370314.0,6
SR,281224.0,2
MH,75125.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3620712.0,25
M-SC,1391331.0,15
X-MC,1296029.0,16
X-SC,973358.0,8
X-LC,871520.0,11
H-MC,404548.0,3
L-SC,269365.0,3
M-LC,111538.0,1
H-LC,66257.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1272843.0      6
           AR         884164.0      5
M-SC       XY24       792921.0      6
H-SC       XR         769643.0      7
X-SC       X40N       502910.0      3
X-MC       X40        449094.0      7
           XY24       384932.0      3
X-LC       X40        350537.0      5
H-SC       OX40N      337713.0      4
M-SC       OX40N      304434.0      5
X-SC       XY24       291566.0      3
H-SC       SR         281224.0      2
X-MC       X40N       272933.0      4
X-LC       XY24       232392.0      2
H-MC       AR         210412.0      2
X-LC       X40N       198381.0      2
H-MC       XY24       194136.0      1
X-MC       XY25       189070.0      2
L-SC       XR         181484.0      2
X-SC       X40        178882.0      2
M-SC       XR         169516.0      2
           AR         124460.0      2
M-LC       XR         111538.0      1
X-LC       XY25        90210.0      2
L-SC       OX40N       87881.0      1
H-SC       MH          75125.0      1
H-LC       AR          66257.0      1
L-MC       XR          53669.0      1
M-MC       XY25        51891.0      1
L-LC       XY25        39143.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 38.0 seconds
